In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

class style :
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
    
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# check python bit version
import platform
print(platform.architecture())

!pip install jupyternotify
import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

!pip install email-notify-magic
%load_ext email_notify_magic

('64bit', 'WindowsPE')


<IPython.core.display.Javascript object>

In [2]:
%%notify
%%time

Review= pd.read_excel(r"C:\\Users\\rishabh.pandey\\Downloads\\Tasks\\Task11_Products(5Apr)\Products.xlsx", sheet_name=3, header=0)
Review= Review[['ProductID', 'ReviewText', 'ReviewStar']]
# Review['ReviewText']=Review['ReviewText'].apply(str).apply(lambda x: clean(x))
# Review.to_csv(r"C:\\Users\\rishabh.pandey\\Downloads\\Tasks\\Task11_Products(5Apr)\Review.csv")
print(Review.shape)

(50887, 3)
Wall time: 30.9 s


<IPython.core.display.Javascript object>

## Cleaing: Code

In [3]:
%%notify
%%time

from nltk.corpus import wordnet
from nltk import word_tokenize

def clean(text):
    
    text_token = word_tokenize(text)
    text_token2 = [x.lower() for x in text_token]
    
    # Mapping Social-media shortcuts
    shortcuts = {'u': 'you','lol': 'laughing out loud', 'k': 'okay', 'm': 'am', 'b4': 'before',
                'wlcm': 'welcome', 'bc': 'because', '<3': 'love', 'xoxo': 'love','gr8': 'great', 'bday': 'birthday',
                 'awsm': 'awesome', 'gud': 'good', 'h8': 'hate','irl': 'in real life', 'yolo': 'you only live once','btw': 'by the way'}
    nsw=pd.Series(text_token)
    words=nsw.map(shortcuts).fillna(nsw)
    
    #Get an integrated list of listed_words (words like yolo must be combined, You Only Live Once :)
    words= [word.split(' ') for word in words]    
    import itertools
    words= list(itertools.chain.from_iterable(words))
    
    # lemmatize the words with appropriate POS Tag
    def pos_tag(word):  
        tag= nltk.pos_tag([word])[0][1][0].upper()
        dic= {"J": wordnet.ADJ,"N": wordnet.NOUN,"V": wordnet.VERB,"R": wordnet.ADV}
        return dic.get(tag, wordnet.NOUN)
    words = [WordNetLemmatizer().lemmatize(word, pos=pos_tag(word)) for word in words] 
    words2=[word for word in words if nltk.pos_tag([word])[0][1] not in ['VB','VBG','VBD','VBN','VBP','VBZ','CC','MD','IN']]
    
    '''# perform stemming for non lemmatized words (like 'cleaning')
    from nltk.stem.snowball import SnowballStemmer
    
    englishStemmer=SnowballStemmer("english")
    words=[englishStemmer.stem(word) for  word in words]'''
    
    '''words=[word.encode('utf-8') for word in words]
    words=[hnspl.suggest(word)[0] if hnspl.spell(word)==False else word for word in words]'''
    
    clean_text=" ".join(words2)
    return clean_text

Wall time: 1.44 s


<IPython.core.display.Javascript object>

In [15]:
#!pip install stanza --user
import stanza
# stanza.download('en') # download English model

2021-04-19 19:14:35 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-04-19 19:14:35 INFO: Use device: cpu
2021-04-19 19:14:35 INFO: Loading: tokenize
2021-04-19 19:14:35 INFO: Loading: pos
2021-04-19 19:14:36 INFO: Loading: lemma
2021-04-19 19:14:36 INFO: Loading: depparse
2021-04-19 19:14:37 INFO: Loading: sentiment
2021-04-19 19:14:38 INFO: Loading: ner
2021-04-19 19:14:39 INFO: Done loading processors!


## Stanford&Stanza: ABSA

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize, sent_tokenize 

nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
stop_words = set(stopwords.words('english'))

nlp = stanza.Pipeline('en') # initialize English neural pipeline

In [31]:
def ABSA(txt):
    
    txt = txt.lower() # LowerCasing the given Text
    sentList = nltk.sent_tokenize(txt) # Splitting the text into sentences

    fcluster = []
    totalfeatureList = []
    finalcluster = []
    dic = {}

    for line in sentList:
        newtaggedList = []
        txt_list = nltk.word_tokenize(line) # Splitting up into words
        taggedList = nltk.pos_tag(txt_list) # Doing Part-of-Speech Tagging to each word

        newwordList = []
        flag = 0
        for i in range(0,len(taggedList)-1):
            if(taggedList[i][1]=="NN" and taggedList[i+1][1]=="NN"): # If two consecutive words are Nouns then they are joined together
                newwordList.append(taggedList[i][0]+taggedList[i+1][0])
                flag=1
            else:
                if(flag==1):
                    flag=0
                    continue
                newwordList.append(taggedList[i][0])
                if(i==len(taggedList)-2):
                    newwordList.append(taggedList[i+1][0])

        finaltxt = ' '.join(word for word in newwordList) 
        new_txt_list = nltk.word_tokenize(finaltxt)
        #wordsList = [w for w in new_txt_list if not w in stop_words]
        taggedList = nltk.pos_tag(new_txt_list)

        doc = nlp(finaltxt) # Object of Stanza NLP Pipeleine
        
        # Getting the dependency relations betwwen the words
        dep_node = []
        for dep_edge in doc.sentences[0].dependencies:
            dep_node.append([dep_edge[2].text, dep_edge[0].id, dep_edge[1]])

        # Coverting it into appropriate format
        for i in range(0, len(dep_node)):
            if (int(dep_node[i][1]) != 0):
                dep_node[i][1] = newwordList[(int(dep_node[i][1]) - 1)]

        featureList = []
        categories = []
        for i in taggedList:
            if(i[1]=='JJ' or i[1]=='NN' or i[1]=='JJR' or i[1]=='NNS' or i[1]=='RB'):
                featureList.append(list(i)) # For features for each sentence
                totalfeatureList.append(list(i)) # Stores the features of all the sentences in the text
                categories.append(i[0])

        for i in featureList:
            filist = []
            for j in dep_node:
                if((j[0]==i[0] or j[1]==i[0]) and (j[2] in ["nsubj", "acl:relcl", "obj", "dobj", "agent", "advmod", "amod", "neg", "prep_of", "acomp", "xcomp", "compound"])):
                    if(j[0]==i[0]):
                        filist.append(j[1])
                    else:
                        filist.append(j[0])
            fcluster.append([i[0], filist])
            
    for i in totalfeatureList:
        dic[i[0]] = i[1]
    
    for i in fcluster:
        if(dic[i[0]]=="NN"):
            finalcluster.append(i)
        
    return(finalcluster)

## Test 

In [33]:
txt = "The Sound Quality is great but the battery is bad. Also, the Camera is pathetic and screen is very terrible!"
ABSA(txt)

[['soundquality', ['great']],
 ['battery', ['bad']],
 ['camera', ['pathetic']],
 ['screen', ['terrible']]]